# Processamento de Vídeo utilizando o OpenCV

In [5]:
import cv2
import numpy as np

### 1. Leitura e processamento de um arquivo de vídeo 

In [9]:
# Abre arquivo de vídeo
vcap = cv2.VideoCapture('video.mp4')
while True:
    # Obtém próximo frame do vídeo. ret indica se o frame
    # foi lido com sucesso, curr_frame possui a imagem
    ret, curr_frame = vcap.read()
    if ret==True:
        # Aqui podemos aplicar métodos de processamento de imagem 
        # na imagem curr_frame. Por exemplo, transformar a imagem
        # para nível de cinza
        curr_frame_g = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        curr_frame_g = np.dstack((curr_frame_g, curr_frame_g, curr_frame_g))
        img = np.concatenate((curr_frame, curr_frame_g), axis=1)
        cv2.imshow('frame', img)
        
    # cv2.waitKey(x) faz o programa esperar x milisegundos para que uma tecla 
    # seja digitada. Dependendo da plataforma, o resultado dessa função pode 
    # ser um número com mais de 8 bits. É comum que uma operação bitwise and 
    # seja feita entre o resultado dessa função e o número 0xff (11111111), 
    # de forma que apenas os 8 primeiros bits sejam mantidos. A tecla esc 
    # corresponde ao valor 27
    key = cv2.waitKey(1) 
    if key == 27:
        break

# Libera o arquivo
vcap.release()
# Fecha a janela
cv2.destroyAllWindows()
# Para garantir que a janela será fechada
cv2.waitKey(1)

-1

### 2. Leitura, processamento e escrita de um novo arquivo de vídeo

A operação de escrita de vídeo pode ser problemática devido ao uso de codecs de vídeo

In [3]:
def green_boost(img):
    
    green_channel = img[:,:,1]
    new_green_channel = np.power(green_channel, 0.5)
    new_green_channel_normalized = 255*new_green_channel/np.max(new_green_channel)
    new_green_channel_normalized = np.round(new_green_channel_normalized).astype(np.uint8)
    new_img = img.copy()
    new_img[:,:,1] = new_green_channel_normalized
    
    return new_img

vcap = cv2.VideoCapture('video.mp4')

# Obtém informações sobre o vídeo. 
fps = vcap.get(cv2.CAP_PROP_FPS)
w = int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Obtém o código associado com um dado codec de compressão de vídeo. MJPG
# tende a funcionar em diferentes plataformas, mas é meio ruim. Uma boa
# opção é o 'H264', mas é preciso que ele esteja instalado no sistema
codec = cv2.VideoWriter_fourcc(*'MJPG')

# Cria arquivo de saída
video_writer = cv2.VideoWriter('output.mp4', codec, fps, (w,h))

while True:
    ret, curr_frame = vcap.read()
    if ret==True:
        new_frame = green_boost(curr_frame)        
        video_writer.write(new_frame)
    else:
        break
        
vcap.release()
video_writer.release()

### 3. Leitura e processamento utilizando uma câmera

In [10]:
import scipy.ndimage as ndi

def process_frame(img):
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = img_gray.astype(float)
    grad_mag = ndi.gaussian_gradient_magnitude(img_gray, sigma=2)
    grad_mag_norm = grad_mag - np.min(grad_mag)
    grad_mag_norm = 255*grad_mag_norm/np.max(grad_mag_norm)
    grad_mag_norm = np.round(grad_mag_norm).astype(np.uint8)    
    return grad_mag_norm
    
# Abre dispositivo de captura. O parâmetro 0 faz com que
# a primeira câmera encontrada seja utilizada
vcap = cv2.VideoCapture(0)

while True:
    ret, curr_frame = vcap.read()
    if ret==True:
        # Redimensionamos a imagem para diminuir o custo computacional
        # necessário para processar o frame
        resized_frame = cv2.resize(curr_frame, (640, 360))
        processed_frame = process_frame(resized_frame)
               
        cv2.imshow('frame', processed_frame)
    else:
        print("Frame not available")
        
    # Esperar 33 milisegundos significa mostrar 30 imagens por segundo (1/0.033).
    # Portanto, o FPS (frames per second) do vídeo mostrado será aproximadamente 30.
    # O valor não é exatamente 30 porque um tempo é gasto no processamento do frame
    key = cv2.waitKey(33) & 0xff

    if key == 27:
        break

vcap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1